# A glacier change projection with GCM data

In this example, we illustrate how to do a typical "projection run", i.e. using GCM data. Here we will first use already bias-corrected CMIP6 data from [ISIMIP3b](https://www.isimip.org/gettingstarted/isimip3b-bias-adjustment), but also show alternatives using the original CMIP5 or CMIP6 data. 

There are three important steps:
- download the OGGM pre-processed directories containing a pre-calibrated and spun-up glacier model
- download the projections data (and bias correct them in the case of CMIP5 or CMIP6)
- simulate their future evolution from the present day state to the end of the century (2020-2100)

In [ ]:
# Libs
import xarray as xr
import matplotlib.pyplot as plt

# Locals
import oggm.cfg as cfg
from oggm import utils, workflow, tasks
from oggm.shop import gcm_climate

## Pre-processed directories

Let's do a run for two Himalayan glaciers: Baltoro and Chogo.

In [ ]:
# Initialize OGGM and set up the default run parameters
cfg.initialize(logging_level='WARNING')

# Local working directory (where OGGM will write its output)
cfg.PATHS['working_dir'] = utils.gettempdir('OGGM_gcm_run', reset=True)

# Add additional outputs for the maps below
cfg.PARAMS['store_model_geometry'] = True
cfg.PARAMS['store_fl_diagnostics'] = True

# We use a different evolution model because we want to use centerlines (more on that later the week)
cfg.PARAMS['evolution_model'] = 'FluxBased'

# RGI glaciers: Baltoro and Chogo
rgi_ids = utils.get_rgi_glacier_entities(['RGI60-14.06794', 'RGI60-14.04638'])

# Go - get the pre-processed glacier directories
# You have to explicitly indicate the url from where you want to start from
base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.1/centerlines/W5E5/'
gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=4, prepro_base_url=base_url)

## The `_historical` runs

The level 4 files now come with a pre-computed model run from the RGI outline date to the last possible date given by the historical climate data. In case of the new default climate dataset [GSWP3_W5E5](https://www.isimip.org/gettingstarted/input-data-bias-adjustment/details/80/), this is until the end of 2019, so the volume is computed until January 1st, 2020. These files are stored in the directory with a `_historical` suffix. Let's compile them into a single file for our two glaciers: 

In [ ]:
ds = utils.compile_run_output(gdirs, input_filesuffix='_historical')
ds.volume.plot(hue='rgi_id');

Each RGI glacier has an "inventory date", the time at which the ouline is valid. It can change between glaciers and this is why the two timeseries start at a different date:

In [ ]:
gdirs[0].rgi_date, gdirs[1].rgi_date

One thing to remember here is that although we try hard to avoid spin-up issues, the glacier after the inversion is not in a perfect dynamical state. Some variable (in particular glacier length) might need some years to adjust. Later this week, we talk about ways to deal with this problem (dynamic spinup). For now, these files are perfect for our purpose, since we plan to start our simulation in 2020.

## Download and process GCM data from ISIMIP3b (bias-corrected CMIP6)

A typical use case for OGGM will be to use climate model output (here bias-corrected CMIP6 GCMs from [ISIMIP3b](https://www.isimip.org/gettingstarted/isimip3b-bias-adjustment/)). We use some of the files [we mirrored in Bremen](https://cluster.klima.uni-bremen.de/~oggm/cmip6/isimip3b/flat/monthly/) here, but you can use whichever you want. From ISIMIP3b, we have 5 GCMs and 3 SSPs on the cluster. You can find more information on the [ISIMIP website](https://www.isimip.org/gettingstarted/isimip3b-bias-adjustment). Let's download the data:

In [ ]:
# you can choose one of these 5 different GCMs:
# 'gfdl-esm4_r1i1p1f1', 'mpi-esm1-2-hr_r1i1p1f1', 'mri-esm2-0_r1i1p1f1' ("low sensitivity" models, within typical ranges from AR6)
# 'ipsl-cm6a-lr_r1i1p1f1', 'ukesm1-0-ll_r1i1p1f2' ("hotter" models, especially ukesm1-0-ll)
member = 'mri-esm2-0_r1i1p1f1' 

for ssp in ['ssp126', 'ssp370','ssp585']:
    # bias correct them
    workflow.execute_entity_task(gcm_climate.process_monthly_isimip_data, gdirs, 
                                 ssp = ssp,
                                 # gcm member -> you can choose another one
                                 member=member,
                                 # recognize the climate file for later
                                 output_filesuffix=f'_ISIMIP3b_{member}_{ssp}'
                                 );

The advantage of using ISIMIP3b data is that they have been bias-corrected by the ISIMIP consortium. Since we are using the [W5E5](https://www.isimip.org/gettingstarted/input-data-bias-adjustment/details/78/) dataset as baseline historical data in OGGM v1.6, there is no need for us to bias correct any further. If you want to bias-correct yourself the projections and want to have a larger variety of GCMs, you can also use the original CMIP5 or CMIP6 GCMs: more information [below](#Download-and-process-GCM-data-from-CMIP5-or-CMIP6).

If you want to know which historical data you are using, you can ask OGGM:

In [ ]:
gdirs[0].get_climate_info()

## Projection runs 

We now run OGGM under various scenarios **and start from the end year of the historical run**:

In [ ]:
for ssp in ['ssp126', 'ssp370', 'ssp585']:
    rid = f'_ISIMIP3b_{member}_{ssp}'
    workflow.execute_entity_task(tasks.run_from_climate_data, gdirs,
                                 climate_filename='gcm_data',  # use gcm_data, not climate_historical
                                 climate_input_filesuffix=rid,  # use the chosen scenario
                                 init_model_filesuffix='_historical',  # this is important! Start from 2020 glacier
                                 output_filesuffix=rid,  # recognize the run for later
                                );

## Plot model output 

You can look at the model output as the total glacier change:

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
# Pick some colors for the lines
color_dict={'ssp126':'blue', 'ssp370':'orange', 'ssp585':'red'}
for ssp in ['ssp126','ssp370', 'ssp585']:
    rid = f'_ISIMIP3b_{member}_{ssp}'
    # Compile the output into one file
    ds = utils.compile_run_output(gdirs, input_filesuffix=rid)
    # Plot it
    ds.isel(rgi_id=0).volume.plot(ax=ax1, label=ssp, c=color_dict[ssp]);
    ds.isel(rgi_id=1).volume.plot(ax=ax2, label=ssp, c=color_dict[ssp]);
plt.legend();

Or look at the evolution in a more graphical way along the main flowline, again using xarray functionalities:

In [ ]:
# here we use the plotting package seaborn to get nicer colors
import seaborn as sns
colors = sns.color_palette('rocket', 4)

In [ ]:
# depending on the glaciers the number of centerlines is different, here we want to get the main one
def get_main_flowline_id(gdir, rid):
    f = gdir.get_filepath('fl_diagnostics', filesuffix=rid)
    with xr.open_dataset(f) as ds:
        # We use the "base" grouped dataset to learn about the flowlines
        fl_ids = ds.flowlines.data
    return fl_ids[-1]

In [ ]:
ssp = 'ssp585'

rid = f'_ISIMIP3b_{member}_{ssp}'

# this with comment is here to change the colors of the lines
with plt.rc_context({'axes.prop_cycle': plt.cycler(color=colors)}):
    # here the actual plot starts
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
    with xr.open_dataset(gdirs[0].get_filepath('fl_diagnostics', filesuffix=rid), group=f'fl_{get_main_flowline_id(gdirs[0], rid)}') as ds:
        (ds.bed_h + ds.sel(time=[2021, 2040, 2060, 2080]).thickness_m).plot(ax=ax1, hue='time')
        ds.bed_h.plot(ax=ax1, c='k')
    with xr.open_dataset(gdirs[1].get_filepath('fl_diagnostics', filesuffix=rid), group=f'fl_{get_main_flowline_id(gdirs[1], rid)}') as ds:
        (ds.bed_h + ds.sel(time=[2021, 2040, 2060, 2080]).thickness_m).plot(ax=ax2, hue='time')
        ds.bed_h.plot(ax=ax2, c='k')

    # finally we add titles to the plots 
    ax1.set_title(f'{gdirs[0].rgi_id}, {gdirs[0].name}');
    ax2.set_title(f'{gdirs[1].rgi_id}, {gdirs[1].name}');

Or plot it on a map using oggm plotting functions. This (currently) only works for centerlines and not elevation bands, we will cover the differences between them later the week, but if you are interested you can have a look in the [documentation](https://docs.oggm.org/en/stable/flowlines.html#glacier-flowlines). In short, elevation bands are faster to create and more robust, but you loose geographical information needed for the plots below.

In [ ]:
from oggm import graphics

ssp = 'ssp585'
rid = f'_ISIMIP3b_{member}_{ssp}'

gdir = gdirs[1]
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14, 6))
graphics.plot_modeloutput_map(gdir, filesuffix=rid, modelyr=2021, ax=ax1, vmax=150)
graphics.plot_modeloutput_map(gdir, filesuffix=rid, modelyr=2050, ax=ax2, vmax=150)
graphics.plot_modeloutput_map(gdir, filesuffix=rid, modelyr=2100, ax=ax3, vmax=150)
plt.tight_layout();

**That's it! Your OGGM projection and different ways of visualizing.**

## Download and process GCM data from CMIP5 or CMIP6

ISIMIP data is very useful because it is bias corrected. Furthermore, it offers daily data (which we will soon make available in OGGM).

But you may want a higher diversity of models or scenarios: for this, you may also use the CMIP5 or CMIP5 GCMs directly. These need to be bias-corrected first to the applied baseline historical data (see [process_gcm_data](https://docs.oggm.org/en/stable/generated/oggm.tasks.process_gcm_data.html#oggm.shop.gcm_climate.process_gcm_data). This relatively simple bias-correction is automatically done by `process_cmip_data` and is very important, as the model is very sensitive to temperature variability.
- CMIP5 has 4 different RCP scenarios and a variety of GCMs, online you can find them [here](https://cluster.klima.uni-bremen.de/~oggm/cmip5-ng). The above mentioned storage contains information about the data, [how to cite them](https://cluster.klima.uni-bremen.de/~oggm/cmip5-ng/ABOUT) and [tabular summaries](https://cluster.klima.uni-bremen.de/~oggm/cmip5-ng/gcm_table.html) of the available GCMs. 
- CMIP6 has 4 different SSP scenarios, see [this table](https://cluster.klima.uni-bremen.de/~oggm/cmip6/gcm_table.html) for a summary of available GCMs. There are even some CMIP6 runs that go until [2300](https://cluster.klima.uni-bremen.de/~oggm/cmip6/gcm_table_2300.html).

> Note, that the CMIP5 and CMIP6 files are much larger than the ISIMIP3b files. This is because we use a simple processing trick for the ISIMIP3b GCM files as we only save the glacier gridpoints, instead of  each longitude and latitude. 

**Therefore: run the following code only if it is ok to download a few gigabytes of data:** 

In [ ]:
bp = 'https://cluster.klima.uni-bremen.de/~oggm/cmip5-ng/pr/pr_mon_CCSM4_{}_r1i1p1_g025.nc'
bt = 'https://cluster.klima.uni-bremen.de/~oggm/cmip5-ng/tas/tas_mon_CCSM4_{}_r1i1p1_g025.nc'

color_dict_rcp={'rcp26':'blue', 'rcp45':'violet', 'rcp85':'red'}

# Download and bias correct the data
for rcp in ['rcp26', 'rcp45',  'rcp85']: # 'rcp60' would also be available
    # Download the files
    ft = utils.file_downloader(bt.format(rcp))
    fp = utils.file_downloader(bp.format(rcp))
    # bias correct them
    workflow.execute_entity_task(gcm_climate.process_cmip_data, gdirs, 
                                 filesuffix='_CMIP5_CCSM4_{}'.format(rcp),  # recognize the climate file for later
                                 fpath_temp=ft,  # temperature projections
                                 fpath_precip=fp,  # precip projections
                                 );

# Run OGGM
for rcp in ['rcp26', 'rcp45',  'rcp85']: #'rcp60',
    rid = '_CMIP5_CCSM4_{}'.format(rcp)
    workflow.execute_entity_task(tasks.run_from_climate_data, gdirs, ys=2020, 
                                 climate_filename='gcm_data',  # use gcm_data, not climate_historical
                                 climate_input_filesuffix=rid,  # use the chosen scenario
                                 init_model_filesuffix='_historical',  # this is important! Start from 2020 glacier
                                 output_filesuffix=rid,  # recognize the run for later
                                );

# Plot
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
for rcp in ['rcp26', 'rcp45', 'rcp85']: #'rcp60',
    rid = '_CMIP5_CCSM4_{}'.format(rcp)
    ds = utils.compile_run_output(gdirs, input_filesuffix=rid)
    ds.isel(rgi_id=0).volume.plot(ax=ax1, label=rcp, c=color_dict_rcp[rcp]);
    ds.isel(rgi_id=1).volume.plot(ax=ax2, label=rcp, c=color_dict_rcp[rcp]);
plt.legend();

Now, the same for CMIP6 but instead of RCPs, now SSPs and again with another GCM:

(**Attention! This may take some time ...**) Uncomment the code below to run it.

In [ ]:
# bp = 'https://cluster.klima.uni-bremen.de/~oggm/cmip6/GCM/CESM2/CESM2_{}_r1i1p1f1_pr.nc'
# bt = 'https://cluster.klima.uni-bremen.de/~oggm/cmip6/GCM/CESM2/CESM2_{}_r1i1p1f1_tas.nc'

# # Download and bias correct the data
# for ssp in ['ssp126', 'ssp585']:  # Removed 'ssp245', 'ssp370' because the files are large!
#     # Download the files
#     ft = utils.file_downloader(bt.format(ssp))
#     fp = utils.file_downloader(bp.format(ssp))
#     # bias correct them
#     workflow.execute_entity_task(gcm_climate.process_cmip_data, gdirs, 
#                                  #year_range=('1979', '2014'),
#                                  filesuffix='_CMIP6_CESM2_{}'.format(ssp),  # recognize the climate file for later
#                                  fpath_temp=ft,  # temperature projections
#                                  fpath_precip=fp,  # precip projections
#                                  );

# # Run OGGM
# for ssp in ['ssp126', 'ssp585']:
#     rid = '_CMIP6_CESM2_{}'.format(ssp)
#     workflow.execute_entity_task(tasks.run_from_climate_data, gdirs, ys=2020, 
#                                  climate_filename='gcm_data',  # use gcm_data, not climate_historical
#                                  climate_input_filesuffix=rid,  # use the chosen scenario
#                                  init_model_filesuffix='_historical',  # this is important! Start from 2020 glacier
#                                  output_filesuffix=rid,  # recognize the run for later
#                                 );

# # Plot
# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
# for ssp in ['ssp126', 'ssp585']:
#     rid = '_CMIP6_CESM2_{}'.format(ssp)
#     ds = utils.compile_run_output(gdirs, input_filesuffix=rid)
#     ds.isel(rgi_id=0).volume.plot(ax=ax1, label=ssp, c=color_dict[ssp]);
#     ds.isel(rgi_id=1).volume.plot(ax=ax2, label=ssp, c=color_dict[ssp]);

# plt.legend();

## What's next?

- Have a first look at other [OGGM tutorials](https://oggm.org/tutorials/master/notebooks/welcome.html)
- Have a look at the Documentaion of OGGM [OGGM documentation](https://docs.oggm.org)